In [1]:
import pandas as pd 
import os
from pulp import *

In [2]:
os.chdir("C:/Users/zuhre.b/OneDrive - Procter and Gamble/Desktop/IE313_Assignment1")

In [3]:
single70 = pd.read_excel("ProductInfo.xlsx",sheet_name='Single70')
double70 = pd.read_excel("ProductInfo.xlsx",sheet_name='Double70')
double80 = pd.read_excel("ProductInfo.xlsx",sheet_name='Double80')
total_const = pd.read_excel("ProductInfo.xlsx",sheet_name='Total Constraints')

In [4]:
#Standart time to produce 1 unit
single70_stime = 1.65
double70_stime = 2.15
double80_stime = 2.68

#Max Overtime/RegularTime
max_overtime_const = 0.25

#Overtime cost increase
overtime_penalty = 1.5

#Initial Inventories
single70_inv = 6500
double70_inv = 12000
double80_inv = 3000

***Parameters***

In [5]:
months = list(single70['Months'])

products= list(["Single70","Double70","Double80"])
unitcosts70=list(single70['Production Cost ($/unit)'])
unitcostd70=list(double70['Production Cost ($/unit)'])
unitcostd80=list(double80['Production Cost ($/unit)'])
forecast70=list(single70['Sales Forecast(Units)'])
forecastd70=list(double70['Sales Forecast(Units)'])
forecastd800=list(double80['Sales Forecast(Units)'])
CostRegularMan=list(total_const["Cost of Regular Man-Hour"])
AvailableManHours=list(total_const["Available Man-Hours"])
InventoryCarryRate=list(total_const["Inventory Carry Rate"])

param={}
ays70={}
ayd70={}
ayd80={}
for i in range(len(months)):
    ays70[months[i]]={"forecast" :forecast70[i] , "unit cost" : unitcosts70[i], "CostRegularMan": CostRegularMan[i], "AvailableManHours" : AvailableManHours[i],"InventoryCarryRate" : InventoryCarryRate[i] }
    ayd70[months[i]]={"forecast" :forecastd70[i] , "unit cost" : unitcostd70[i], "CostRegularMan": CostRegularMan[i], "AvailableManHours" : AvailableManHours[i],"InventoryCarryRate" : InventoryCarryRate[i] }
    ayd80[months[i]]={"forecast" :forecastd800[i] , "unit cost" : unitcostd80[i],"CostRegularMan": CostRegularMan[i], "AvailableManHours" : AvailableManHours[i],"InventoryCarryRate" : InventoryCarryRate[i] }

for i in range(len(months)):
    param["Single70"]=ays70
    param["Double70"]=ayd70
    param["Double80"]=ayd80
    


In [6]:
#Decision Variables

prod_rate = LpVariable.dicts("prod_rate",((month,product) for month in months for product in products),lowBound=0,cat='Continuous')

regular_work = LpVariable.dicts("regular_work",((month,product) for month in months for product in products),lowBound=0,cat='Continuous')

overtime_work = LpVariable.dicts("overtime_work",((month,product) for month in months for product in products),lowBound = 0,cat='Continuous')

end_inventory = LpVariable.dicts("end_inventory",((month,product) for month in months for product in products),lowBound=0,cat='Continuous')

extra_work = LpVariable.dicts("extra_work",((month,product) for month in months for product in products),lowBound=0,cat='Continuous')

In [7]:
#Initialize Problem

prob = LpProblem("Supply_Chain_Management",LpMinimize)

# Objective Function


In [8]:
prob += lpSum([param[prod][month]["unit cost"]*prod_rate[(month,prod)] + param[prod][month]["unit cost"]*param[prod][month]["InventoryCarryRate"]*end_inventory[(month,prod)] + regular_work[(month,prod)]*param[prod][month]["CostRegularMan"] + param[prod][month]["CostRegularMan"]*overtime_work[(month,prod)]*overtime_penalty + 38*extra_work[(month,prod)] for month in months for prod in products])

In [10]:
for prod in products:
    for i in range(1,len(months)): 
        prob += end_inventory[(months[i-1],prod)] + prod_rate[(months[i],prod)] - end_inventory[(months[i],prod)] >= param[prod][months[i]]["forecast"]

In [11]:
prob += single70_inv + prod_rate[(months[0],products[0])] - end_inventory[(months[0],products[0])] >= param[products[0]][months[0]]["forecast"]
prob += double70_inv + prod_rate[(months[0],products[1])] - end_inventory[(months[0],products[1])] >= param[products[1]][months[0]]["forecast"]
prob += double80_inv + prod_rate[(months[0],products[2])] - end_inventory[(months[0],products[2])] >= param[products[2]][months[0]]["forecast"]

In [12]:
for month in ["Jan","Feb","Mar","Apr","May","Sep","Oct","Nov","Dec"]:
    for prod in products:
        prob += extra_work[(month,prod)] == 0

In [13]:
for month in months:
        prob += lpSum([regular_work[(month,prod)] for prod in products]) <= param[products[0]][month]["AvailableManHours"]

In [14]:
for month in months:
        prob += lpSum([overtime_work[(month,prod)] for prod in products]) <= lpSum([regular_work[(month,prod)] for prod in products])*0.25

In [15]:
for month in months:
        prob += prod_rate[(month,products[0])]*single70_stime <= regular_work[(month,products[0])] + overtime_work[(month,products[0])] + extra_work[(month,products[0])]
        prob += prod_rate[(month,products[1])]*double70_stime <= regular_work[(month,products[1])] + overtime_work[(month,products[1])] + extra_work[(month,products[1])]
        prob += prod_rate[(month,products[2])]*double80_stime <= regular_work[(month,products[2])] + overtime_work[(month,products[2])] + extra_work[(month,products[2])]

In [16]:
prob.solve()

1

In [17]:
print("Status:", LpStatus[prob.status])

Status: Optimal


In [18]:
for v in prob.variables():
    print(v.name, "=", v.varValue)

end_inventory_('Apr',_'Double70') = 0.0
end_inventory_('Apr',_'Double80') = 3716.4179
end_inventory_('Apr',_'Single70') = 4660.6061
end_inventory_('Aug',_'Double70') = 34806.977
end_inventory_('Aug',_'Double80') = 0.0
end_inventory_('Aug',_'Single70') = 30000.0
end_inventory_('Dec',_'Double70') = 0.0
end_inventory_('Dec',_'Double80') = 0.0
end_inventory_('Dec',_'Single70') = 0.0
end_inventory_('Feb',_'Double70') = 0.0
end_inventory_('Feb',_'Double80') = 20716.418
end_inventory_('Feb',_'Single70') = 0.0
end_inventory_('Jan',_'Double70') = 0.0
end_inventory_('Jan',_'Double80') = 3481.3433
end_inventory_('Jan',_'Single70') = 25000.0
end_inventory_('Jul',_'Double70') = 0.0
end_inventory_('Jul',_'Double80') = 0.0
end_inventory_('Jul',_'Single70') = 59000.0
end_inventory_('Jun',_'Double70') = 0.0
end_inventory_('Jun',_'Double80') = 0.0
end_inventory_('Jun',_'Single70') = 87000.0
end_inventory_('Mar',_'Double70') = 0.0
end_inventory_('Mar',_'Double80') = 11216.418
end_inventory_('Mar',_'Singl

In [19]:
value(prob.objective)

394054275.8433751